In [ ]:
import os
#ncore = "1"
#os.environ["OMP_NUM_THREADS"] = ncore
#os.environ["OPENBLAS_NUM_THREADS"] = ncore
#os.environ["MKL_NUM_THREADS"] = ncore
#os.environ["VECLIB_MAXIMUM_THREADS"] = ncore
#os.environ["NUMEXPR_NUM_THREADS"] = ncore
import sys
import json
import xarray as xr
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path
from functools import partial
import warnings

warnings.filterwarnings('ignore')

In [ ]:
sys.path.append('/pscratch/sd/w/wmtsai/featenv_analysis/runscripts/')
os.chdir('/pscratch/sd/w/wmtsai/featenv_analysis/config/config_track_env')
from feature_environment_module_CMIP import *
from multiprocessing import Pool

# read feature and variable settings from .json files
feature_json = open('feature_list.jsonc')
variable_json = open('varible_list.CMIP6.jsonc')
feature_settings = json.load(feature_json)
variable_settings = json.load(variable_json)

# call the feature-environemnt module
featenv = ds_feature_environment()
print('version: ', featenv.__version__)
featenv.name = feature_settings['feature'][0]['name']
featenv.feature_data_sources = feature_settings['feature'][0]['feature_sources']
featenv.environmental_data_sources = feature_settings['feature'][0]['feature_environment_sources']
featenv.track_frequency = feature_settings['feature'][0]['track_frequency']
featenv.env_frequency = feature_settings['feature'][0]['track_frequency']
featenv.feature_track = eval(feature_settings['feature'][0]['is_feature_track'])
featenv.feature_mask = eval(feature_settings['feature'][0]['is_feature_mask'])
featenv.box_size_degree = int(feature_settings['feature'][0]['box_size_degree'])
featenv.is_same_latlon = eval(feature_settings['feature'][0]['is_same_latlon'])

In [ ]:
# matching default ERA-5
if featenv.is_same_latlon:
    var_name = variable_settings['variable_inputs'][0]['varname_infile']
    var_dir = Path(variable_settings['variable_inputs'][0]['var_dir'])
    files = list(var_dir.glob('*.nc'))
    ds = xr.open_dataset(files[0]) # read the data to get lat-lon information
    try:
        featenv.lon_env = ds.lon.values
        featenv.lat_env = ds.lat.values
    except:
        featenv.lon_env = ds.longitude.values
        featenv.lat_env = ds.latitude.values
    print('Feature and Environment in the same lat/lon coordinate...')
    print('longitude grid points:', len(featenv.lon_env))
    print('latitude grid points:', len(featenv.lat_env))

else:
    print('Default coordinated lat/lon: ERA-5, 0.25-deg.')
    featenv.lon_env = np.arange(0,360,0.25)
    featenv.lat_env = np.arange(-90,90.25,0.25)
    
    print('longitude grid points:', len(featenv.lon_env))
    print('latitude grid points:', len(featenv.lat_env))

In [ ]:
# locate and read the preprocessed track file
featenv.track_data =  xr.open_dataset(feature_settings['feature'][0]['track_data'], decode_times=True)
coords_track = []
vars_track = []
for i in featenv.track_data.dims:
    coords_track.append(i)
for i in featenv.track_data.keys():
    vars_track.append(i)

a = set(coords_track)
b = set(['tracks','time'])
c = set(vars_track)
d = set(['base_time','meanlon','meanlat'])
if (a & b) == {'time','tracks'} and (c & d) == {'base_time','meanlon','meanlat'}:
    print('Track data...ready: {}'.format(feature_settings['feature'][0]['track_data']))
else:
    print('track input error')
    #sys.exit('Incorret input format...Check the input file') # exit due to an incorrect track file

In [ ]:
tmp = xr.open_dataset(feature_settings['feature'][0]['track_data'], decode_times=True)
str(tmp.isel(time=0, tracks=0).base_time.values)[:4]

In [ ]:
temp = xr.open_dataset('/pscratch/sd/w/wmtsai/featenv_analysis/dataset/MCS_FLEXTRKR_tropics/1950/environment_catalogs/VARS_2D/MCS_FLEXTRKR_tropics_rlut.merged.nc')
temp2 = xr.open_dataset('/pscratch/sd/w/wmtsai/featenv_analysis/dataset/MCS_FLEXTRKR_tropics/1950/environment_catalogs/VARS_2D/MCS_FLEXTRKR_tropics_pr.merged.nc')
temp3 = xr.open_dataset('/pscratch/sd/w/wmtsai/featenv_analysis/dataset/MCS_FLEXTRKR_tropics/1950/environment_catalogs/VARS_2D/MCS_FLEXTRKR_tropics_cloudtracknumber.merged.nc')

In [ ]:
temp.isel(tracks=1,time=1).base_time.values

In [ ]:
#temp.isel(tracks=1, time=0).rlut.plot()
#Tf = tb(a+b*Tb) where a = 1.228 and b = -1.106e-3
temp_rlut = temp.isel(tracks=1, time=0).rlut
a = 1.228
b = -1.106e-3
sigma = 5.67e-8
tf = (temp_rlut/sigma)**0.25
tb = (-a + np.sqrt(a**2 + 4*b*tf))/(2*b)

tb.plot.contour(levels=[240,250], colors=['m'], zorder=3)
tb.plot.contour(levels=[250,255,260], colors=['k'],zorder=2)

temp2.isel(tracks=1, time=0).pr.plot.contour(cmap='jet')
(temp3.isel(tracks=1, time=0).cloudtracknumber.fillna(0)).plot()

In [ ]:
np.unique(temp3.cloudtracknumber.isel(tracks=1,time=0))

In [ ]:
# check MCS data
test = xr.open_dataset('/global/cfs/cdirs/m4374/input_datasets/year2_EC-Earth3P-HR_highressst-present_r3i1f1p1/CMIP6HighResMIPEC-Earth-ConsortiumEC-Earth3P-HRhighresSST-presentr3i1p1f1grv20190509/rlut/rlut_E3hr_EC-Earth3P-HR_highresSST-present_r3i1p1f1_gr_201101010000-201112312100.nc')
lon_env = test.lon.values
lat_env = test.lat.values

mcs_dir = Path('/global/cfs/cdirs/m4374/catalogues/raw_catalogue_files/highresmip/CMIP6HighResMIPEC-Earth-ConsortiumEC-Earth3P-HRhighresSST-presentr3i1p1f1grv20190509/MCS/mcstracking/19500101.0000_19501231.2100')
mcs_orig = xr.open_dataset(mcs_dir / 'mcstrack_19500110_0600.nc')
mcs_orig.coords['lon'] = lon_env
mcs_orig.coords['lat'] = lat_env

In [ ]:
mcs_orig.isel(time=0).cloudtracknumber.sel(lon = temp.isel(tracks=1,time=1).lon.values, 
                                           lat = temp.isel(tracks=1,time=1).lat.values).plot()

In [ ]:
np.unique(mcs_orig.isel(time=0).cloudtracknumber.sel(lon = temp.isel(tracks=1,time=0).lon.values, 
                                           lat = temp.isel(tracks=1,time=0).lat.values, method='nearest'))